In [ ]:
# Import libraries
from google.colab import files
from zipfile import ZipFile
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Upload zip file
uploaded = files.upload()

In [ ]:
# Specify zip file name
file_name = "NewDataset.zip"

# Extract dataset
with ZipFile(file_name, 'r') as zip:
    zip.printdir() # Print all files in the zip file
    print('Extracting all files...')
    zip.extractall("LeafDisease") # Extracts all files into a new folder
    print('Dataset extracted successfully!')

File Name                                             Modified             Size
NewDataset/                                    2025-11-06 22:44:24            0
NewDataset/Training/                           2025-11-06 23:24:02            0
NewDataset/Training/Apple___Apple_scab/        2025-11-12 11:13:34            0
NewDataset/Training/Apple___Apple_scab/023123cb-7b69-4c9f-a521-766d7c8543bb___FREC_Scab 3487.JPG 2025-11-05 00:49:50         8897
NewDataset/Training/Apple___Apple_scab/1a304331-98b5-473f-bbb1-c33b8441052a___FREC_Scab 3296.JPG 2025-11-05 00:49:50        15567
NewDataset/Training/Apple___Apple_scab/1a41bab0-45e0-4dda-a798-9bf4a998f1b6___FREC_Scab 3450.JPG 2025-11-05 00:49:50        15466
NewDataset/Training/Apple___Apple_scab/1cb869ea-0a4c-47d7-9def-a88c16b72ddc___FREC_Scab 3350.JPG 2025-11-05 00:49:50        14854
NewDataset/Training/Apple___Apple_scab/1cdf2779-8cb1-4ae4-be70-b0eeb1abb0f8___FREC_Scab 3370.JPG 2025-11-05 00:49:50        12335
NewDataset/Training/Apple___Ap

In [ ]:
import os

# Define dataset directories
train_path = "LeafDisease/NewDataset/Training"
test_path = "LeafDisease/NewDataset/Validation"

# Data Cleaning
def clean_dataset(dataset_path, dataset_name="Dataset"):
    total_images = 0
    corrupted_images = 0

    print(f"\nChecking dataset: {dataset_name}")

    for class_name in os.listdir(dataset_path):
        class_path = os.path.join(dataset_path, class_name)
        if not os.path.isdir(class_path):
            continue

        for filename in os.listdir(class_path):
            file_path = os.path.join(class_path, filename)

            # Try reading the image
            img = cv2.imread(file_path)
            if img is None:
                os.remove(file_path)
                corrupted_images += 1
                continue

            total_images += 1

    print("\n--- Summary ---")
    print(f"Total valid images in {dataset_name}: {total_images}")
    print(f"Total corrupted images removed: {corrupted_images}")

# Run cleaning on both datasets
clean_dataset(train_path, "Training")
clean_dataset(test_path, "Validation")


Checking dataset: Training

--- Summary ---
Total valid images in Training: 1329
Total corrupted images removed: 0

Checking dataset: Validation

--- Summary ---
Total valid images in Validation: 570
Total corrupted images removed: 0


In [ ]:
def segment_leaf(image, disease_type):

    # Convert to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define HSV ranges for each disease type
    disease_hsv_ranges = {
        "Strawberry___Leaf_scorch": {"green": ([25,20,20],[100,255,255]), "brown": ([5,0,10],[25,255,180])},
        "Tomato___Septoria_leaf_spot": {"green": ([25,20,20],[100,255,255]), "brown": ([0,10,20],[20,255,180])},
        "Orange___Haunglongbing_(Citrus_greening)": {"green": ([30,20,20],[90,255,255]), "yellow": ([15,20,40],[45,255,255])},
        "Blueberry___healthy": {"green": ([25,20,20],[100,255,255])},
        "Apple___Black_rot": {"green": ([25,20,20],[100,255,255]), "brown": ([0,10,20],[25,255,180])},
        "Tomato___Leaf_Mold": {"green": ([25,20,20],[100,255,255]), "brown": ([0,10,20],[25,255,180])},
        "Tomato___Tomato_mosaic_virus": {"green": ([25,20,20],[100,255,255]), "yellow": ([15,20,40],[45,255,255])},
        "Pepper,_bell___healthy": {"green": ([25,20,20],[100,255,255])},
        "Soybean___healthy": {"green": ([25,20,20],[100,255,255])},
        "Tomato___Target_Spot": {"green": ([25,20,20],[100,255,255]), "brown": ([0,10,20],[25,255,180])},
        "Corn_(maize)___Northern_Leaf_Blight": {"green": ([25,20,20],[100,255,255]), "brown": ([0,10,20],[25,255,180])},
        "Potato___Late_blight": {"green": ([10,40,40],[100,230,230]), "brown": ([0,10,20],[25,130,100])},
        "Corn_(maize)___Common_rust_": {"green": ([25,20,20],[100,255,255]), "brown": ([0,10,20],[25,255,180])},
        "Tomato___Tomato_Yellow_Leaf_Curl_Virus": {"green": ([20,30,30],[120,255,255]), "yellow": ([0,10,20],[50,150,30])},
        "Grape___Leaf_blight_(Isariopsis_Leaf_Spot)": {"green": ([10,20,20],[80,255,255]), "brown": ([0,0,0],[10,20,20])},
        "Grape___healthy": {"green": ([25,20,20],[100,255,255])},
        "Peach___Bacterial_spot": {"green": ([10,20,20],[100,200,200]), "brown": ([10,20,10],[20,100,100])},
        "Tomato___Bacterial_spot": {"green": ([20,30,30],[120,250,250]), "brown": ([0,10,20],[25,150,30])},
        "Grape___Black_rot": {"green": ([20,0,0],[100,200,255]), "brown": ([0,0,0],[0,50,50])},
        "Apple___healthy": {"green": ([25,20,20],[100,255,255])},
        "Tomato___healthy": {"green": ([25,20,20],[100,255,255])},
        "Squash___Powdery_mildew": {"green": ([25,20,20],[100,255,255]), "white": ([0,0,200],[180,40,255])},
        "Pepper,_bell___Bacterial_spot": {"green": ([10,20,20],[100,230,230]), "brown": ([0,10,20],[25,130,100])},
        "Peach___healthy": {"green": ([25,20,20],[100,255,255])},
        "Apple___Cedar_apple_rust": {"green": ([25,20,20],[100,255,255]), "orange": ([5,50,50],[25,255,255])},
        "Potato___healthy": {"green": ([25,20,20],[100,255,255])},
        "Strawberry___healthy": {"green": ([25,20,20],[100,255,255])},
        "Cherry_(including_sour)___Powdery_mildew": {"green": ([25,20,20],[100,255,255]), "white": ([0,0,200],[180,40,255])},
        "Tomato___Spider_mites Two-spotted_spider_mite": {"green": ([25,20,20],[100,255,255]), "yellow": ([15,20,40],[45,255,255])},
        "Tomato___Late_blight": {"green": ([25,20,20],[100,255,255]), "brown": ([0,10,20],[25,255,180])},
        "Potato___Early_blight": {"green": ([25,20,20],[100,255,255]), "brown": ([0,10,20],[25,255,180])},
        "Tomato___Early_blight": {"green": ([25,20,20],[100,255,255]), "brown": ([0,10,20],[25,255,180])},
        "Raspberry___healthy": {"green": ([25,20,20],[100,255,255])},
        "Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot": {"green": ([25,20,20],[100,255,255]), "brown": ([0,10,20],[25,255,180])},
        "Apple___Apple_scab": {"green": ([25,20,20],[100,255,255]), "brown": ([0,10,20],[25,255,180])},
        "Corn_(maize)___healthy": {"green": ([25,20,20],[100,255,255])},
        "Cherry_(including_sour)___healthy": {"green": ([25,20,20],[100,255,255])},
        "Grape___Esca_(Black_Measles)": {"green": ([10,20,20],[80,255,255]), "brown": ([0,10,20],[20,50,50])}
    }

    # Create mask for each disease type
    mask_total = None
    for color, (lower, upper) in disease_hsv_ranges[disease_type].items():
        lower = np.array(lower)
        upper = np.array(upper)
        mask = cv2.inRange(hsv, lower, upper)
        mask_total = mask if mask_total is None else cv2.bitwise_or(mask_total, mask)

    # Morphological operations to clean mask
    kernel = np.ones((5,5), np.uint8)
    mask_total = cv2.morphologyEx(mask_total, cv2.MORPH_CLOSE, kernel)  # fills small holes
    mask_total = cv2.morphologyEx(mask_total, cv2.MORPH_OPEN, kernel)   # removes small noise

    # Apply the mask to the original image
    result = np.zeros_like(image)                  # create black background
    result[mask_total != 0] = image[mask_total != 0]  # keep only leaf area

    return result, mask_total


In [ ]:
from tqdm import tqdm

input_dir = "/content/LeafDisease/NewDataset/Validation"
output_dir = "LeafDiseaseSegmentation/Validation"

# Loop through each disease class folder
for folder in tqdm(os.listdir(input_dir)):
    folder_path = os.path.join(input_dir, folder)
    save_path = os.path.join(output_dir, folder)
    os.makedirs(save_path, exist_ok=True)

    # Loop through images in this class
    for file in os.listdir(folder_path):
        if file.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(folder_path, file)
            img = cv2.imread(img_path)
            if img is None:
                continue

            # Segment based on disease type
            segmented, mask = segment_leaf(img, disease_type=folder)
            save_file = os.path.join(save_path, file)

            cv2.imwrite(save_file, segmented)


100%|██████████| 38/38 [00:02<00:00, 15.34it/s]


In [ ]:
import shutil

# Create a zip file of the output folder
shutil.make_archive("LeafDiseaseSegmentation", 'zip', "LeafDiseaseSegmentation/Validation")

# Download to your computer
files.download("LeafDiseaseSegmentation.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>